In [89]:
#import externally developed packages
import os
import sys
import vcfpy
import re
import pandas as pd

## Import a .vcf file and convert it into a dataframe with a unique identifier for each SNP

In [122]:
##import vcf

#specify path of vcf
vcf_path = '/Users/tristanpwdennis/Software/seqtoolbox/analysis-scripts/vcf/vcf/test.vcf'

#import vcf as reader object
reader = vcfpy.Reader.from_path(vcf_path)

# Build header
header = pd.DataFrame.transpose(pd.DataFrame(['#CHROM', 'POS', 'REF', 'ALT'] + reader.header.samples.names))

# Design matrix in list format
X_list = []  

#convert vcf convert to dataframe - extract GT field, chrom, pos ref and alt data
for record in reader:
    if not record.is_snv():
        continue
    line = [record.CHROM, record.POS, record.REF]
    line += [alt.value for alt in record.ALT]
    line += [call.data.get('GT') or './.' for call in record.calls]
    X_list.append(line)
    df = pd.DataFrame(X_list)

#stick header onto the genotype frame
df=header.append(df) 

#make added row into the df header
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [129]:
##vcf to colony input
#make unique genotype id from chrom, pos ref and alt
id=df["#CHROM"].map(str) + df["POS"].map(str) + df["REF"].map(str) + df["ALT"].map(str)
#add new header
df.insert(0, 'id', id)
#drop chrom pos ref and alt fields
gt=df.drop(['#CHROM', 'POS', 'REF', 'ALT'], axis=1)

#specify mapping (from genotype values to codes)
di = {
    "./.": 0,
    "0/0": 1,
    "0/1": 2,
    "1/0": 3,
    "1/1": 4,
}  

#rename genotypes to be codes following mapping above
gt=gt.replace(di)

#transpose into individuals as columns and genotypes as rows
gt=gt.transpose()

In [132]:
gt.head()

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,,,,,,,,,,,,,,,,,,,,,
id,OVOC_OO_0003271374CT,OVOC_OO_0003271375GA,OVOC_OO_0003271571AC,OVOC_OO_0003271612TC,OVOC_OO_0003271629GT,OVOC_OO_0003271766AT,OVOC_OO_0003271785TC,OVOC_OO_0003271846AG,OVOC_OO_0003271958AG,OVOC_OO_0003272094AG,...,OVOC_OO_00002410258TG,OVOC_OO_00002410264GA,OVOC_OO_00002410268TC,OVOC_OO_00002410299TA,OVOC_OO_00002411392AC,OVOC_OO_00002411432TA,OVOC_OO_00002411443TC,OVOC_OO_00002411561CT,OVOC_OO_00002411823AT,OVOC_OO_00002412197GT
ind0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,2,2,1,2,2
ind1,2,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,2,2,2,2,2
ind2,1,2,1,1,1,2,1,1,1,1,...,2,2,1,2,1,1,2,2,4,2
ind3,1,1,1,2,1,1,1,2,2,1,...,2,1,2,2,1,4,2,2,2,2
